## Mixing System

Problem statement: The figure below shows a set of well-mixed mixing tanks. All the streams contain a binary mixture of substance $X$ and substance $Y$. Steams $A$, $B$ and $C$ are fed into the system from an upstream process.

Tanks $1$ and $2$ are drained by the force of gravity (assume flow is proportional to level), while the pump attached to the tank $3$ output is sized such that the level in tank $3$ does not affect the flow rate through the pump.

You may assume that the valves in lines $G$, $H$, $J$ and $L$ can manipulate those flows directly.

The density of substance $X$ is $ρX = 1000 kg/m3$ and the density of substance $Y$ is $ρY = 800 kg/m3.$

![Mixing System](./img/tanksystem.png)

<!-- Continue for chapter 15 -->